In [1]:
import requests
import pandas as pd
import time
from requests.structures import CaseInsensitiveDict
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import psycopg2 as ps

In [2]:
#Keys
CLIENT_ID = 'xxx'
SECRET_KEY = 'xxx'
ACCESS_TOKEN = 'xxx'

In [3]:
def get_data(df):
    
    #stack api connection variables to get 100 most viewed streams at the moment
    url = 'https://api.twitch.tv/helix/streams?first=100'
    headers = CaseInsensitiveDict()
    headers["Authorization"] = f"Bearer {ACCESS_TOKEN}"
    headers["Client-Id"] = CLIENT_ID

    
    resp = requests.get(url, headers=headers).json()
    
    #loop for each json collection - append everything to a pd dataframe
    time.sleep(1)
    for stream in resp['data']:
        user_name = stream['user_name']
        game_name = stream['game_name']
        transmission_type = stream['type']
        title = stream['title']
        viewer_count = stream['viewer_count']
        started_at = stream['started_at']
        started_at = datetime.strptime(started_at, "%Y-%m-%dT%H:%M:%S%z")
        language = stream['language']
        is_mature = stream['is_mature']

        df = df.append(
            {'user_name': user_name,
             'game_name': game_name,
             'type': transmission_type,
             'title': title,
             'viewer_count': viewer_count,
             'started_at': started_at,
             'language': language,
             'is_mature': is_mature},
            ignore_index = True
        )
        
    return df

In [4]:
#create an empty df for right columns
df = pd.DataFrame(columns=[
        'user_name',
        'game_name',
        'type',
        'title',
        'viewer_count',
        'started_at',
        'language',
        'is_mature'
    ])

#get streaming data and append it to the empty df
df = get_data(df)

In [5]:
df

,user_name,game_name,type,title,viewer_count,started_at,language,is_mature
0,ibai,Just Chatting,live,LUIS ENRIQUE DEBUTA COMO STREAMER EL SELECCION...,48441,2022-11-18 18:23:07+00:00,es,False
1,HasanAbi,Just Chatting,live,TWITTER IS ACTUALLY IMPLODING!!!!! OKBUDDYFRIDAY!,40382,2022-11-18 19:34:55+00:00,en,False
2,ElisaEsports,Counter-Strike: Global Offensive,live,ENCE vs. Astralis | Quarter-Finals | Elisa Mas...,30312,2022-11-18 13:43:42+00:00,en,False
3,Elraenn,Grounded,live,yayak,27891,2022-11-18 17:13:47+00:00,tr,True
4,Nmplol,World of Warcraft,live,[DROPS] OTK WORLD OF WARCRAFT 3v3 DRAFT INVITA...,26280,2022-11-18 16:01:56+00:00,en,False
...,...,...,...,...,...,...,...,...
94,ungespielt,World of Warcraft,live,DROPS AKTIV! - !Subathon - #ungeklickt & WoW !...,4506,2022-11-18 19:03:00+00:00,de,False
95,Cydonia_Chiara,Pokémon Scarlet/Violet,live,"BLIND RUN Pokémon Scarlatto e Violetto #2: ""Me...",4495,2022-11-18 13:59:29+00:00,it,False
96,Ray,Pokémon Scarlet/Violet,live,!giveaway !store | POKEMON SCARLET/VIOLET IS H...,4494,2022-11-18 17:45:19+00:00,en,False
97,VooDooSh,The Dark Pictures Anthology: The Devil in Me,live,Новый Кино-Хоррор от Dark Pictures! The Devil ...,4442,2022-11-18 20:14:37+00:00,ru,False


In [ ]:
### Database connection section

In [6]:
# Check if we connected succesfully to the cloud db
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
        
    except ps.OperationalError as e:
        raise e
    else:
            print('Connected')
    return conn

In [18]:
# Create table in the cloud db

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS stream (
                        user_name varchar(255) not null,
                        game_name varchar(255) not null,
                        type varchar(10) not null,
                        title varchar(225) not null,
                        viewer_count integer not null,
                        started_at date not null,
                        language varchar(2) not null,
                        is_mature varchar(10) not null
                        )""")
    curr.execute(create_table_command)

In [15]:
# insert data into postgres table
def insert_into_table(curr, user_name, game_name, type, title, viewer_count, started_at, language, is_mature):
    insert_into_streams = ("""INSERT INTO stream (user_name, game_name, type, title, viewer_count, started_at, language, is_mature)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s);""")

    row_to_insert = (user_name, game_name, type, title, viewer_count, started_at, language, is_mature)

    curr.execute(insert_into_streams, row_to_insert)

In [9]:
#Push data from pandas dataframe to the newly connected db 
def append_from_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['user_name'],
                                row['game_name'],
                                row['type'],
                                row['title'],
                                row['viewer_count'],
                                row['started_at'],
                                row['language'],
                                row['is_mature'])

In [21]:
# DB connection variables
host_name = 'database-project-twitch.xxx'
dbname = 'x'
port = '5432'
username = 'postgres'
password = 'x'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Connected


In [22]:
curr = conn.cursor()

In [23]:
create_table(curr)

In [24]:
append_from_df_to_db(curr, df)

In [26]:
conn.commit()

In [30]:
#If no results then failed, else data pushed successfully

curr.fetchall()

[('HasanAbi',
  'Just Chatting',
  'live',
  'TWITTER IS ACTUALLY IMPLODING!!!!! OKBUDDYFRIDAY!',
  40382,
  datetime.date(2022, 11, 18),
  'en',
  'false'),
 ('ElisaEsports',
  'Counter-Strike: Global Offensive',
  'live',
  'ENCE vs. Astralis | Quarter-Finals | Elisa Masters Espoo 2022',
  30312,
  datetime.date(2022, 11, 18),
  'en',
  'false'),
 ('Elraenn',
  'Grounded',
  'live',
  'yayak',
  27891,
  datetime.date(2022, 11, 18),
  'tr',
  'true'),
 ('Nmplol',
  'World of Warcraft',
  'live',
  '[DROPS] OTK WORLD OF WARCRAFT 3v3 DRAFT INVITATIONAL FT ASMONGOLD SODAPOPPIN & NMPLOL',
  26280,
  datetime.date(2022, 11, 18),
  'en',
  'false'),
 ('GRONKH',
  'Just Chatting',
  'live',
  '#FREiAB18 // !horde2 !ff8',
  25356,
  datetime.date(2022, 11, 18),
  'de',
  'true'),
 ('LIRIK',
  'Warhammer 40,000: Darktide',
  'live',
  'Me Smash',
  24970,
  datetime.date(2022, 11, 18),
  'en',
  'false'),
 ('juansguarnizo',
  'Goat Simulator 3',
  'live',
  'VIERNES TRAVIESO +18 PARA ADULTOS 